In [1]:
from functools import partial
import itertools

from functools32 import lru_cache
"""
Corpus example taking from PhD thesis

abbac
abccba
bbac
cba """
D=[["a","b","b","a","c"],["a","b","c","c","b","a"],["b","b","a","c"],["c","b","a"]]
N=7

In [2]:
#@lru_cache(maxsize=500000)
def ngrams(input, n):
  if n<=0:
    return {}
  output = {}
  for i in range(len(input)-n+1):
    g = ' '.join(input[i:i+n])
    output.setdefault(g, 0)
    output[g] += 1
  return output 

def add(x,y):
    return { k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y) }

In [3]:
def memo(f):
    d = {}
    def func(arg):
        if arg not in d:
            d[arg]=f(arg)
        return d[arg]
    return func

def memo2(f):
    d = {}
    def func(arg1,arg2):
        key = str(arg1) + str(arg2)
        if arg not in d:
            d[key]=f(arg1,arg2)
        return d[key]
    return func


#@memo2
def buildProducts(W,n):
    return map(" ".join,itertools.product(W, repeat= n))

#@memo
def buildVocabulary(D):
    res ={}
    for sentence in D:
        #TODO: introduce caching via dictionary
        tmp = ngrams(sentence,1)
        res = add(tmp,res)
#    print res,res.keys()
    return res.keys()

#@lru_cache(maxsize=500)
def buildAbsoluteCounter(D,n):
    W = buildVocabulary(D)
    W_n = buildProducts(W,n)
    
    
    def absoluteCounts(s):
        res = {}
        if s in W_n:
            #print "din't hit cache"
            for sentence in D:
                #TODO: introduce caching via dictionary
                tmp = ngrams(sentence,n)
                res = add(tmp,res)
            if s in res:
                return res[s]
            return 0
    return absoluteCounts
    
def buildNGramMLE(D,n,absoluteCounts):
    W = buildVocabulary(D)
    W_n = buildProducts(W,n)

    def totalSum(baseSet):
        return sum([absoluteCounts(el) for el in baseSet])
    
    def estimator(s):
        return float(absoluteCounts(s))/totalSum(W_n)    

    return estimator

def buildContCounter(D,n,absoluteCounts):
    W = buildVocabulary(D)
    W_n_1 = buildProducts(W,n-1)
    
    def contCounter(s):
        res = 0
        for w in W:
            if absoluteCounts(" ".join([w,s])) > 0:
                res = res + 1
        return res
    return contCounter

def buildMarkovModel(D,n,h,absoluteCounts):
    W = buildVocabulary(D)
    
    W_full = []
    for w in W:
        W_full.append(" ".join([h,w]))
    
    def totalSum(baseSet):
        return sum(map(absoluteCounts,baseSet))
    
    
    def markovModel(w):
        if totalSum(W_full) > 0:
            return float(absoluteCounts(" ".join([h,w])))/totalSum(W_full)
        else: #backoff order! TODO: we could have added the order to which backoff took place in particular backoff needs no backoff weights because everything is working nicely. I am just destroing the markov assumption for histories where it doesn't hold
            freq = memo(buildAbsoluteCounter(D,n))            
            x = buildMarkovModel(D,n-1," ".join(h.split(" ")[1:]),freq)
            return x(w)
    return markovModel

In [4]:
MLE={}
for i in range(1,N):
    res = 0
    freq = memo(buildAbsoluteCounter(D,i))
    mle = memo(buildNGramMLE(D,i,freq))
    for s in buildProducts(buildVocabulary(D),i):
        res = res + mle(s)
    print "n = ", i , " sum = ", res
    MLE[i] = mle

n =  1  sum =  1.0
n =  2  sum =  1.0
n =  3  sum =  1.0
n =  4  sum =  1.0
n =  5  sum =  1.0
n =  6  sum =  1.0


In [5]:
CC = {}
for i in range(1,N-1):
    res = 0
    freq = memo(buildAbsoluteCounter(D,i+1))
    cc = memo(buildContCounter(D,i,freq))
    mle = memo(buildNGramMLE(D,i,cc))
    for s in buildProducts(buildVocabulary(D),i):
        #print cc(s)
        res = res + mle(s)
    print "n = ", i , " sum = ", res
    CC[i] = mle

n =  1  sum =  1.0
n =  2  sum =  1.0
n =  3  sum =  1.0
n =  4  sum =  1.0
n =  5  sum =  1.0


In [6]:
def testNGram(s):
    n = len(s.split(" "))
    print "s= ",s, " n= ", n, " MLE= ", MLE[n](s), " CC= ", CC[n](s)
for s in buildProducts(buildVocabulary(D),1):
    testNGram(s)

# TODO: CONJECTURE: I start doubting weather it makes really sense to backoff to continuation counts in Kneser Ney Smoothing.
# If the history has not been seen why would I count in how many different contexts the history appeared?

s=  a  n=  1  MLE=  0.333333333333  CC=  0.142857142857
s=  c  n=  1  MLE=  0.277777777778  CC=  0.428571428571
s=  b  n=  1  MLE=  0.388888888889  CC=  0.428571428571


In [19]:
#TODO how to make this in a way to not compute it over and over again
def l(m,rho):
    if m<1 or rho <=0 or rho>=1:
        raise ("argument in wrong range, m should be an int bigger than 0 and rho a real between 0 and 1. rho was: "+str(rho) + " m was: " + str(m))
    return rho*pow(1-rho,m-1)
    return d[m]

@memo
def queryNGramLM(query):
    words = query.split(" ")
    m = len(words)
    freq = memo(buildAbsoluteCounter(D,m))
    mle = memo(buildNGramMLE(D,m,freq))
    return mle(query)*l(m,0.2)

@memo
def queryMarkov0(query):
    freq = memo(buildAbsoluteCounter(D,1))
    mle = memo(buildNGramMLE(D,1,freq))
    words = query.split(" ")
    m = len(words)
    res = l(m,0.2)
    for word in words:
        res = res * mle(word)
    return res
N = 5
globalSum = 0
for i in range(1,N):
    localSum = 0
    for s in buildProducts(buildVocabulary(D),i):
        localSum = localSum + queryNGramLM(s)
    globalSum = globalSum + localSum
    print "n = ", i , " globalSum = " , globalSum ," localSum = ", localSum, " lambda: ", l(i,0.2)
print queryNGramLM("a b")

n =  1  globalSum =  0.2  localSum =  0.2  lambda:  0.2
n =  2  globalSum =  0.36  localSum =  0.16  lambda:  0.16
n =  3  globalSum =  0.488  localSum =  0.128  lambda:  0.128
n =  4  globalSum =  0.5904  localSum =  0.1024  lambda:  0.1024
0.0228571428571


In [14]:
globalSum = 0
for i in range(1,N):
    localSum = 0
    for s in buildProducts(buildVocabulary(D),i):
        localSum = localSum + queryMarkov0(s)
    globalSum = globalSum + localSum
    print "n = ", i , " globalSum = " , globalSum ," localSum = ", localSum, " lambda: ", l(i,0.2)
print queryNGramLM("a a a")

n =  1  globalSum =  0.2  localSum =  0.2  lambda:  0.2
n =  2  globalSum =  0.36  localSum =  0.16  lambda:  0.16
n =  3  globalSum =  0.488  localSum =  0.128  lambda:  0.128
n =  4  globalSum =  0.5904  localSum =  0.1024  lambda:  0.1024
0.0


In [9]:
i=4
for h in buildProducts(buildVocabulary(D),i):
    freq = memo(buildAbsoluteCounter(D,i+1))
    mm = buildMarkovModel(D,i,h,freq)
    res = 0
    for s in buildProducts(buildVocabulary(D),1):
        res = res + mm(s)
#        print "P("+s+"|"+h+") = ", mm(s)
    print "h= ",h ," n = ", i , " sum = ", res



h=  a a a a  n =  4  sum =  1.0
h=  a a a c  n =  4  sum =  1.0
h=  a a a b  n =  4  sum =  1.0
h=  a a c a  n =  4  sum =  1.0
h=  a a c c  n =  4  sum =  1.0
h=  a a c b  n =  4  sum =  1.0
h=  a a b a  n =  4  sum =  1.0
h=  a a b c  n =  4  sum =  1.0
h=  a a b b  n =  4  sum =  1.0
h=  a c a a  n =  4  sum =  1.0
h=  a c a c  n =  4  sum =  1.0
h=  a c a b  n =  4  sum =  1.0
h=  a c c a  n =  4  sum =  1.0
h=  a c c c  n =  4  sum =  1.0
h=  a c c b  n =  4  sum =  1.0
h=  a c b a  n =  4  sum =  1.0
h=  a c b c  n =  4  sum =  1.0
h=  a c b b  n =  4  sum =  1.0
h=  a b a a  n =  4  sum =  1.0
h=  a b a c  n =  4  sum =  1.0
h=  a b a b  n =  4  sum =  1.0
h=  a b c a  n =  4  sum =  1.0
h=  a b c c  n =  4  sum =  1.0
h=  a b c b  n =  4  sum =  1.0
h=  a b b a  n =  4  sum =  1.0
h=  a b b c  n =  4  sum =  1.0
h=  a b b b  n =  4  sum =  1.0
h=  c a a a  n =  4  sum =  1.0
h=  c a a c  n =  4  sum =  1.0
h=  c a a b  n =  4  sum =  1.0
h=  c a c a  n =  4  sum =  1.0
h=  c a 

In [20]:
def helperqueryMarkovLM(query,order):
    if order==0:
        freq = memo(buildAbsoluteCounter(D,1))
        mle = memo(buildNGramMLE(D,1,freq))
        return mle(query)
    words = query.split(" ")
    m = len(words)
    
    res = 1
    i = order
    freq = buildAbsoluteCounter(D,order+1)
    for i in range(order, len(words)):
        w = words[i]
        h = " ".join(words[i-order:i])
#        print h,w
        mm = buildMarkovModel(D,order,h,freq)
        res = res * mm(w)
#        print res
#    print "recorsive call: ", " ".join(words[0:order]), order-1, " result from resursive call: ",
    x = helperqueryMarkovLM(" ".join(words[0:order]), order-1)
#    print x
    res = res * x
    return res

def queryMarkovLM(query,order):
    words = query.split(" ")
    m = len(words)
    return l(m,0.2) * helperqueryMarkovLM(query,order) 
    
N = 5
globalSum = 0
for i in range(1,N):
    localSum = 0
    for s in buildProducts(buildVocabulary(D),i):
        localSum = localSum + queryNGramLM(s)
    globalSum = globalSum + localSum
    print "n = ", i , " globalSum = " , globalSum ," localSum = ", localSum, " lambda: ", l(i,0.2)


n =  1  globalSum =  0.2  localSum =  0.2  lambda:  0.2
n =  2  globalSum =  0.36  localSum =  0.16  lambda:  0.16
n =  3  globalSum =  0.488  localSum =  0.128  lambda:  0.128
n =  4  globalSum =  0.5904  localSum =  0.1024  lambda:  0.1024


In [21]:
#TODO how to make this in a way to not compute it over and over again
def l(m,rho):
    if m<1 or rho <=0 or rho>=1:
        raise ("argument in wrong range, m should be an int bigger than 0 and rho a real between 0 and 1. rho was: "+str(rho) + " m was: " + str(m))
    d={i:float(1)/float(6) for i in range(1,7)}
    return d[m]

import math
def calcEntropy():
    Hmm = 0
    Hmm1 = 0
    Hmm2 = 0
    Hmm3=0
    Hng = 0
    
    zmm=0
    zmm1=0
    zmm2=0
    zmm3=0
    zng =0
    N=7
    for i in range(1,N):
        for s in buildProducts(buildVocabulary(D),i):
            mm = queryMarkov0(s)
            ng = queryNGramLM(s)
            mm1 = queryMarkovLM(s,1)
            mm2 = queryMarkovLM(s,2)
            mm3 = queryMarkovLM(s,3)

            if (mm>0):
                Hmm = Hmm - mm*math.log(mm)
            else:
                zmm = zmm+1
            if (mm1>0):
                Hmm1 = Hmm1 - mm1*math.log(mm1)
            else:
                zmm1 = zmm1+1
            if (mm2>0):
                Hmm2 = Hmm2 - mm2*math.log(mm2)
            else:
                zmm2 = zmm2+1
            if (ng>0):
                Hng = Hng - ng*math.log(ng)
            else:
                zng = zng+1
            if (mm3>0):
                Hmm3 = Hmm3 - mm3*math.log(mm3)
            else:
                zmm3 = zmm3+1

            #print s, "       ", "{0:.5f}".format(mm), "{0:.5f}".format(mm1),"{0:.5f}".format(mm2), "{0:.5f}".format(ng)
    maxH = math.log(3+9+27+81+243+729)
    print "Entropy {0:.5f}".format(math.log(3+9+27+81+243+729)), "{0:.5f}".format(Hmm), "{0:.5f}".format(Hmm1),"{0:.5f}".format(Hmm2),"{0:.5f}".format(Hmm3), "{0:.5f}".format(Hng)
    print "E-rel   {0:.5f}".format(1.0), "{0:.5f}".format(Hmm/maxH), "{0:.5f}".format(Hmm1/maxH),"{0:.5f}".format(Hmm2/maxH),"{0:.5f}".format(Hmm3/maxH), "{0:.5f}".format(Hng/maxH)
    print "zeros  ", "0.00000","{0:.4f}".format(zmm), "{0:.4f}".format(zmm1),"{0:.4f}".format(zmm2), "{0:.4f}".format(zmm3), "{0:.4f}".format(zng)
    print "     ", "  equal  ", "MM - 0 ", "MM - 1 ", "MM - 2 ", "MM - 3 ", "ngram Model  " 
calcEntropy()
#result makes sens. the n-gram lm should always produce the lowest entropy value
#real question is: when smoothing the n-gram model will the entropy still be below the markov models
#other question: can an n-gram lm be seen as a \infty-markov language model?

#TODO: FIX: something seems still a little bit wrong. Even though my markov models sum up for MM-3 we have no change in comparison to MM-2

Entropy 6.99577 5.60434 4.85749 3.84245 3.84245 2.80662
E-rel   1.00000 0.80111 0.69435 0.54925 0.54925 0.40119
zeros   0.00000 0.0000 686.0000 1036.0000 1036.0000 1066.0000
        equal   MM - 0  MM - 1  MM - 2  MM - 3  ngram Model  


#Oldstuff not erased yet

In [10]:
#    res = 0
    
    for s in buildProducts(buildVocabulary(D),1):
        res = res + mm(s)
        print "P("+s+"|"+h+") = ", mm(s)
    print "h= ",h ," n = ", i , " sum = ", res
    
    freq = memo(buildAbsoluteCounter(D,m))
    mle = memo(buildNGramMLE(D,m,freq))
    return mle(query)*l(m,0.2)

IndentationError: unexpected indent (<ipython-input-10-d09628c35165>, line 3)

In [65]:
def ngram(n):
    def totalSum(arg,baseSet):
        return sum([arg(el) for el in baseSet])

    def c(s):
        res ={}
        for sentence in D:
            #TODO: introduce caching via dictionary
            tmp = ngrams(" ".join(sentence),n)
            res = add(tmp,res)
        if s in res:
            return res[s]
        return 0

    def uniform(s):
        return 1

    def estimator(f,s):
        return float(f(s))/totalSum(f,buildProducts(buildVocabulary(D),n))    
    return {"a":c,"u":uniform,"ae":partial(estimator, c), "ue":partial(estimator, uniform)}

print ngram(3)["ae"]("a b b")
print buildVocabulary(D)
print buildProducts(buildVocabulary(D),2)

c3("a b b")

0.1
['a', 'c', 'b']
['a a', 'a c', 'a b', 'c a', 'c c', 'c b', 'b a', 'b c', 'b b']


1

In [150]:
def foo(arg):
    print "call"
    return arg* arg
def bar(arg):
    print "call"
    return arg* arg * arg

times = memo(bar)
cachedsquare = memo(foo)

In [151]:
print cachedsquare(2)
print cachedsquare(2)
print cachedsquare(2)

print times(2)
print times(2)
times = memo(bar)
print times(2)


call
4
4
4
call
8
8
call
8


In [167]:
#print order(3)["e"]("a b b",order(3)["a"])
import itertools
print map(" ".join,itertools.product("abc",repeat = 2))

['a a', 'a b', 'a c', 'b a', 'b b', 'b c', 'c a', 'c b', 'c c']


In [ ]:
def c(n,s):
    res ={}
    for sentence in D:
        tmp = ngrams(sentence,n)
        res = add(tmp,res)
    return res[s]
print c(2,)

In [88]:
print l(5,0.5)

0.03125


In [112]:
absoluteCounts = {i:{} for i in range(0,N)}
for s in D:
    for i in range(0,N):
        res = ngrams(s,i)
        absoluteCounts[i] = add(absoluteCounts[i],res)

totalCounts = {i:sum(absoluteCounts[i].values()) for i in range(0,N)}
        
for i in range(0,N):
    print i, totalCounts[i], absoluteCounts[i]

    #TODO: make this more general
ngramModels={i:{k:float(v)/totalCounts[i] for k,v in absoluteCounts[i].iteritems()} for i in range(0,N)}

for i in range(0,N):
    print i, totalCounts[i], ngramModels[i]

0 0 {}
1 18 {'a': 6, 'c': 5, 'b': 7}
2 14 {'a b': 2, 'a c': 2, 'b a': 4, 'b c': 1, 'b b': 2, 'c b': 2, 'c c': 1}
3 10 {'a b b': 1, 'a b c': 1, 'b a c': 2, 'b b a': 2, 'c b a': 2, 'c c b': 1, 'b c c': 1}
4 6 {'a b c c': 1, 'b c c b': 1, 'c c b a': 1, 'b b a c': 2, 'a b b a': 1}
5 3 {'a b c c b': 1, 'b c c b a': 1, 'a b b a c': 1}
6 1 {'a b c c b a': 1}
7 0 {}
